Introduction to Bioinformatics

HomeWork 3

Statistical tests

name: Farzan Rahmani

student id: 403210725



# Gene Expression Analysis Using Statistical Tests

This notebook guides you through analyzing a gene expression dataset to identify significant differences between two groups: **Lung Tumor** and **Normal Lung**. It includes the following key steps:

- **Loading the dataset**: Familiarize yourself with the data structure.
- **Descriptive statistics**: Calculate basic measures such as means and standard deviations.
- **Hypothesis testing**: Use statistical tests (e.g., t-tests) to identify differences between the groups.
- **Multiple testing correction**: Adjust p-values to control the False Discovery Rate.
- **Filtering significant genes**: Identify and prioritize statistically significant genes.

Some sections include **###TO DO tasks**, where you'll complete the code or analysis to achieve specific results.


## Step 1: Load the Dataset

The dataset contains gene expression levels for samples classified as **Lung Tumor** and **Normal Lung**. Each row corresponds to a specific gene, and columns represent expression levels across samples.

In this step, we load the dataset and display the first few rows to understand its structure.

In [3]:
!gunzip /content/GSE10072_series_matrix.txt.gz

In [47]:
!ls

GSE10072_series_matrix.txt  sample_data


In [48]:
### TO DO
import pandas as pd

# Load the dataset (Series Matrix File)

# dataset it located in the following path:
# /content/GSE10072_series_matrix.txt
file_path = '/content/GSE10072_series_matrix.txt'

data = pd.read_csv(file_path, sep="\t", comment='!', index_col=0) # Load the data, skipping metadata lines (! lines)


# Display the first few rows
data.head()

,GSM254625,GSM254626,GSM254627,GSM254628,GSM254629,GSM254630,GSM254631,GSM254632,GSM254633,GSM254634,...,GSM254722,GSM254723,GSM254724,GSM254725,GSM254726,GSM254727,GSM254728,GSM254729,GSM254730,GSM254731
ID_REF,,,,,,,,,,,,,,,,,,,,,
1007_s_at,10.927084,10.416978,10.628538,10.151180,10.988512,10.778205,10.568814,10.479425,10.548843,10.465005,...,10.727493,10.740353,10.527962,10.193615,10.768815,10.467693,10.902778,10.869402,10.292285,10.407218
1053_at,6.895217,6.924856,7.550245,6.699557,6.826031,6.718372,6.739765,6.724615,7.102475,6.627922,...,6.856575,6.737879,6.959032,7.008578,6.740444,6.795750,6.838162,6.628363,6.794050,6.358410
117_at,8.110190,7.760228,7.974676,7.712676,7.775592,7.777087,7.893210,8.058398,8.005438,7.961476,...,7.741922,7.805107,8.093078,8.144760,8.159483,7.855457,8.010428,7.889019,8.163266,7.973844
121_at,9.451286,9.520943,9.807597,9.522087,9.855061,9.861055,10.126183,9.878970,10.110318,10.411478,...,10.089149,10.007059,9.832610,10.107004,9.790645,9.645239,9.871851,9.867988,9.824801,9.850144
1255_g_at,4.814477,4.718640,4.905163,4.818076,4.823958,4.848313,5.125956,5.037979,4.889936,5.177818,...,5.048391,4.758050,4.900470,4.948760,4.609322,4.759571,4.788774,4.967626,4.817474,5.128892


In [49]:
data.shape

(22283, 107)

## Step 2: Assign Sample Groups

The samples are divided into two groups:
- **Normal Lung**: Includes expression levels for normal lung tissue.
- **Lung Tumor**: Includes expression levels for lung tumor tissue.

The corresponding sample names are assigned to these groups based on metadata from the dataset.


In [26]:
## DO NOT Change this cell

normal_lung = [
    "GSM254626", "GSM254628", "GSM254632", "GSM254634", "GSM254635",
    "GSM254638", "GSM254640", "GSM254643", "GSM254644", "GSM254646",
    "GSM254649", "GSM254651", "GSM254653", "GSM254655", "GSM254658",
    "GSM254660", "GSM254662", "GSM254665", "GSM254667", "GSM254669",
    "GSM254671", "GSM254673", "GSM254676", "GSM254677", "GSM254679",
    "GSM254681", "GSM254683", "GSM254685", "GSM254689", "GSM254691",
    "GSM254693", "GSM254695", "GSM254699", "GSM254702", "GSM254703",
    "GSM254706", "GSM254708", "GSM254710", "GSM254711", "GSM254712",
    "GSM254713", "GSM254715", "GSM254717", "GSM254719", "GSM254723",
    "GSM254725", "GSM254727", "GSM254730", "GSM254731"
]

lung_tumor = [
    "GSM254625", "GSM254627", "GSM254629", "GSM254630", "GSM254631",
    "GSM254633", "GSM254636", "GSM254637", "GSM254639", "GSM254641",
    "GSM254642", "GSM254645", "GSM254647", "GSM254648", "GSM254650",
    "GSM254652", "GSM254654", "GSM254656", "GSM254657", "GSM254659",
    "GSM254661", "GSM254663", "GSM254664", "GSM254666", "GSM254668",
    "GSM254670", "GSM254672", "GSM254674", "GSM254675", "GSM254678",
    "GSM254680", "GSM254682", "GSM254684", "GSM254686", "GSM254687",
    "GSM254688", "GSM254690", "GSM254692", "GSM254694", "GSM254696",
    "GSM254697", "GSM254698", "GSM254700", "GSM254701", "GSM254704",
    "GSM254705", "GSM254707", "GSM254709", "GSM254714", "GSM254716",
    "GSM254718", "GSM254720", "GSM254721", "GSM254722", "GSM254724",
    "GSM254726", "GSM254728", "GSM254729"
]

## Step 3: Calculate Descriptive Statistics

For each gene, we calculate:
- **Mean** expression levels for the two groups.
- **Standard Deviation (Std)** to understand the variability within each group.

These statistics provide an initial overview of the data distribution.


In [50]:
### TO DO

# Calculate means for each group
data['Mean_Normal_Lung'] = data[normal_lung].mean(axis=1)
data['Mean_Lung_Tumor'] = data[lung_tumor].mean(axis=1)


# Display the updated DataFrame (mean columns)
data[['Mean_Normal_Lung', 'Mean_Lung_Tumor']]

,Mean_Normal_Lung,Mean_Lung_Tumor
ID_REF,,
1007_s_at,10.309772,10.772500
1053_at,6.736384,6.988408
117_at,8.007275,8.016987
121_at,9.862801,9.878110
1255_g_at,4.857075,4.923281
...,...,...
AFFX-ThrX-5_at,5.238874,5.245832
AFFX-ThrX-M_at,4.578030,4.587327
AFFX-TrpnX-3_at,4.105845,4.114805


In [51]:
### TO DO

# Calculate standard deviations for each group
data['Std_Normal_Lung'] = data[normal_lung].std(axis=1)
data['Std_Lung_Tumor'] = data[lung_tumor].std(axis=1)

# Display the updated DataFrame (std columns)
data[['Std_Normal_Lung', 'Std_Lung_Tumor']]

,Std_Normal_Lung,Std_Lung_Tumor
ID_REF,,
1007_s_at,0.231752,0.364252
1053_at,0.137845,0.270940
117_at,0.463988,0.275375
121_at,0.171649,0.170197
1255_g_at,0.125765,0.138166
...,...,...
AFFX-ThrX-5_at,0.168916,0.134160
AFFX-ThrX-M_at,0.132896,0.129014
AFFX-TrpnX-3_at,0.087820,0.101882


In [52]:
# Display the updated DataFrame (mean columns and std columns)
data[['Mean_Normal_Lung', 'Mean_Lung_Tumor', 'Std_Normal_Lung', 'Std_Lung_Tumor']]

,Mean_Normal_Lung,Mean_Lung_Tumor,Std_Normal_Lung,Std_Lung_Tumor
ID_REF,,,,
1007_s_at,10.309772,10.772500,0.231752,0.364252
1053_at,6.736384,6.988408,0.137845,0.270940
117_at,8.007275,8.016987,0.463988,0.275375
121_at,9.862801,9.878110,0.171649,0.170197
1255_g_at,4.857075,4.923281,0.125765,0.138166
...,...,...,...,...
AFFX-ThrX-5_at,5.238874,5.245832,0.168916,0.134160
AFFX-ThrX-M_at,4.578030,4.587327,0.132896,0.129014
AFFX-TrpnX-3_at,4.105845,4.114805,0.087820,0.101882


## Step 4: Calculate Log2 Fold Change (Log2FC)

Log2 Fold Change $( \log_2 \text{FC} )$ measures the relative difference in expression levels between the **Lung Tumor** and **Normal Lung** groups for each gene. It is calculated as:
$
\text{Log2FC} = \log_2\left(\frac{\text{Mean Lung Tumor}}{\text{Mean Normal Lung}}\right)
$

Genes with a large positive or negative Log2FC indicate stronger differences in expression levels between the two groups.


In [53]:
### TO DO
import numpy as np

# Compute Log2 Fold Change (Log2FC) for each gene
# data['Log2FC'] = np.log2(data['Mean_Lung_Tumor'] / data['Mean_Normal_Lung']) # اگر دیتا ها در مبنای دو نبودند این چنین میشد
# log2(data['Mean_Lung_Tumor'] / data['Mean_Normal_Lung']) --> log2(data['Mean_Lung_Tumor']) - log2(data['Mean_Normal_Lung'])
data['Log2FC'] = data['Mean_Lung_Tumor'] - data['Mean_Normal_Lung'] # این خروجی شبیه خروجی خواسته شده است چرا که دیتا ها خودشون تو مبنای دو هستن

# Display the updated DataFrame with Log2FC (mean columns and Log2FC columns)
data[['Mean_Lung_Tumor', 'Mean_Normal_Lung', 'Log2FC']]

,Mean_Lung_Tumor,Mean_Normal_Lung,Log2FC
ID_REF,,,
1007_s_at,10.772500,10.309772,0.462727
1053_at,6.988408,6.736384,0.252024
117_at,8.016987,8.007275,0.009712
121_at,9.878110,9.862801,0.015309
1255_g_at,4.923281,4.857075,0.066207
...,...,...,...
AFFX-ThrX-5_at,5.245832,5.238874,0.006959
AFFX-ThrX-M_at,4.587327,4.578030,0.009297
AFFX-TrpnX-3_at,4.114805,4.105845,0.008960


In [54]:
### TO DO

# Keep only genes with log2FC > 1 or log2FC < -1
filtered_data = data[(data['Log2FC'] > 1) | (data['Log2FC'] < -1)]

# Display the number of filtered genes
print(f"Number of genes with |Log2FC| > 1: {filtered_data.shape[0]}")

Number of genes with |Log2FC| > 1: 859


In [55]:
### TO DO

# Sort the filtered genes by Log2FC in descending order
filtered_data = filtered_data.sort_values(by='Log2FC', ascending=False)

# Display the genes sorted by Log2FC
filtered_data[['Mean_Lung_Tumor', 'Mean_Normal_Lung', 'Log2FC']]

,Mean_Lung_Tumor,Mean_Normal_Lung,Log2FC
ID_REF,,,
209875_s_at,11.697897,7.333482,4.364415
37892_at,8.718949,5.657426,3.061522
204475_at,7.852579,4.990543,2.862036
206239_s_at,9.403921,6.645521,2.758400
218469_at,8.418868,5.870505,2.548363
...,...,...,...
215454_x_at,6.956141,10.945963,-3.989822
205982_x_at,10.401727,14.396847,-3.995120
211735_x_at,10.466733,14.468768,-4.002035


## Step 5: Perform Hypothesis Testing

We perform an independent two-sample t-test for each gene to compare expression levels between the two groups:
- Null Hypothesis ($H_0$): There is no significant difference in mean expression levels between the groups.
- Alternative Hypothesis ($H_a$): There is a significant difference in mean expression levels between the groups.

The p-values resulting from these tests indicate the statistical significance of the differences.


In [56]:
### TO DO
from scipy.stats import ttest_ind

# Initialize an empty list to store p-values
p_values = []

# Perform t-test for each gene
for index, row in data.iterrows():
    normal_values = row[normal_lung]
    tumor_values = row[lung_tumor]
    t_stat, p_value = ttest_ind(normal_values, tumor_values, equal_var=False)
    p_values.append(p_value)

# Add p-values as a new column in the DataFrame
data['P_Value'] = p_values

# Display the updated DataFrame with P-Value (mean columns and P-value column)
data[['Mean_Normal_Lung', 'Mean_Lung_Tumor', 'P_Value']]

,Mean_Normal_Lung,Mean_Lung_Tumor,P_Value
ID_REF,,,
1007_s_at,10.309772,10.772500,3.167943e-12
1053_at,6.736384,6.988408,1.825086e-08
117_at,8.007275,8.016987,8.979953e-01
121_at,9.862801,9.878110,6.454791e-01
1255_g_at,4.857075,4.923281,1.087906e-02
...,...,...,...
AFFX-ThrX-5_at,5.238874,5.245832,8.163490e-01
AFFX-ThrX-M_at,4.578030,4.587327,7.155850e-01
AFFX-TrpnX-3_at,4.105845,4.114805,6.261823e-01


## Step 6: Adjust P-Values for Multiple Testing

To account for multiple comparisons, we apply the Benjamini-Hochberg procedure to control the false discovery rate (FDR). This ensures that the reported significant genes are less likely to include false positives.

Adjusted p-values (\(q\)-values) are calculated for each gene.


In [57]:
### TO DO
from statsmodels.stats.multitest import multipletests

# Adjust p-values using Benjamini-Hochberg correction
data['Adj_P_Value'] = multipletests(data['P_Value'], method='fdr_bh')[1]


# Display the updated DataFrame with Adj P-Value (mean columns and P-value column and Adj P-Value column)
data[['Mean_Normal_Lung', 'Mean_Lung_Tumor', 'P_Value', 'Adj_P_Value']]

,Mean_Normal_Lung,Mean_Lung_Tumor,P_Value,Adj_P_Value
ID_REF,,,,
1007_s_at,10.309772,10.772500,3.167943e-12,4.047664e-11
1053_at,6.736384,6.988408,1.825086e-08,1.199304e-07
117_at,8.007275,8.016987,8.979953e-01,9.325485e-01
121_at,9.862801,9.878110,6.454791e-01,7.426276e-01
1255_g_at,4.857075,4.923281,1.087906e-02,2.414763e-02
...,...,...,...,...
AFFX-ThrX-5_at,5.238874,5.245832,8.163490e-01,8.746792e-01
AFFX-ThrX-M_at,4.578030,4.587327,7.155850e-01,7.991470e-01
AFFX-TrpnX-3_at,4.105845,4.114805,6.261823e-01,7.260874e-01


## Step 7: Filter and Sort Significant Genes

Genes with adjusted p-values (\(q\)-values) below a significance threshold (e.g., 0.05) are considered statistically significant.
These genes are sorted in ascending order of adjusted p-values to prioritize the most significant results.


In [60]:
### TO DO

# Filter significant genes based on adjusted p-value and Log2FC
# Criteria: Adj P-Value < 0.05 and |Log2FC| > 1
significant_genes = data[(data['Adj_P_Value'] < 0.05) & (data['Log2FC'].abs() > 1)]

# Sort the significant genes by adjusted p-value in ascending order
significant_genes = significant_genes.sort_values(by='Adj_P_Value')

# Display the top 10 significant genes (mean columns and Log2FC	column and P-Value column and Adj P-Value)
significant_genes[['Mean_Lung_Tumor', 'Mean_Normal_Lung', 'Log2FC', 'P_Value', 'Adj_P_Value']].head(10)

,Mean_Lung_Tumor,Mean_Normal_Lung,Log2FC,P_Value,Adj_P_Value
ID_REF,,,,,
209074_s_at,7.451084,10.822883,-3.371799,6.641899e-44,1.480014e-39
204396_s_at,7.583586,9.971658,-2.388072,2.009124e-41,2.238466e-37
204677_at,6.804586,9.424160,-2.619573,1.317136e-40,9.783245e-37
208982_at,9.836642,11.650556,-1.813914,3.357325e-39,1.870282e-35
213103_at,7.630347,8.850167,-1.219820,5.221337e-39,2.326941e-35
208981_at,9.379371,11.068273,-1.688902,7.219182e-39,2.681084e-35
210081_at,7.459343,11.876812,-4.417470,4.769095e-38,1.518139e-34
209875_s_at,11.697897,7.333482,4.364415,1.782532e-37,3.972016e-34
202112_at,9.567466,11.825939,-2.258474,1.464679e-37,3.972016e-34


In [62]:
significant_genes.shape

(859, 114)

In [63]:
significant_genes

,GSM254625,GSM254626,GSM254627,GSM254628,GSM254629,GSM254630,GSM254631,GSM254632,GSM254633,GSM254634,...,GSM254729,GSM254730,GSM254731,Mean_Normal_Lung,Mean_Lung_Tumor,Std_Normal_Lung,Std_Lung_Tumor,Log2FC,P_Value,Adj_P_Value
ID_REF,,,,,,,,,,,,,,,,,,,,,
209074_s_at,8.798452,11.213718,6.573177,11.382607,7.357722,7.996503,7.220028,8.243608,6.525294,10.147571,...,8.910616,11.472560,10.815008,10.822883,7.451084,0.705674,0.756793,-3.371799,6.641899e-44,1.480014e-39
204396_s_at,8.776471,10.623173,6.647853,10.671849,6.775872,8.939531,7.244104,8.762719,6.505241,8.762600,...,8.217307,10.337060,10.357737,9.971658,7.583586,0.515850,0.597344,-2.388072,2.009124e-41,2.238466e-37
204677_at,8.210323,9.806784,7.526138,10.078267,6.517446,7.381125,5.939049,9.115684,5.332512,9.288824,...,7.808221,9.764055,9.902233,9.424160,6.804586,0.587678,0.658071,-2.619573,1.317136e-40,9.783245e-37
208982_at,10.395082,12.189666,8.636143,11.869240,9.977163,10.447010,9.547914,11.120649,9.394159,11.306612,...,10.832153,12.125696,11.866601,11.650556,9.836642,0.396989,0.498926,-1.813914,3.357325e-39,1.870282e-35
213103_at,8.165323,9.160261,7.569503,8.872367,7.449661,7.521799,7.463150,8.453335,7.527500,8.671189,...,8.038184,9.210000,9.091207,8.850167,7.630347,0.287129,0.318038,-1.219820,5.221337e-39,2.326941e-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213432_at,11.303882,6.360045,6.738352,6.116072,10.808133,6.492138,6.798715,7.477874,7.150635,7.059722,...,6.531764,6.252096,6.583040,6.991620,8.202386,0.859568,1.828097,1.210767,2.248864e-05,8.561668e-05
203913_s_at,8.862679,9.012886,6.604695,8.977703,11.096142,5.517124,7.235933,6.767335,9.505086,9.113323,...,8.212926,9.199634,8.775598,8.735059,7.701774,0.989585,1.455168,-1.033285,3.302742e-05,1.226379e-04
209374_s_at,12.742374,8.866113,10.247918,7.727836,11.366013,11.598475,9.505330,10.031624,9.521901,10.495097,...,9.729184,8.474894,6.703889,9.428752,10.574722,1.315039,1.476728,1.145970,4.748817e-05,1.714483e-04
